In [10]:
import cv2
import numpy as np
import imutils
from datetime import datetime as dt
import os
import blend_modes
import re



input_dir = "/Users/phil/Desktop/Working/Digitizing/23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat/run3circles_90-35-4ms"

comp_codec = 'avc1'
raw_codec = 'I420'

fps = 30
vid_res = "1024x1024"
vid_frames = 800

In [133]:
###### USE THIS

def scanDir(directory, template="."):
    vid_list=[]
    for root, dirs, files in os.walk(directory):
        for name in files:
            if name.endswith(".avi") and not re.search(template,name):
                filename = os.path.join(root, name)
                print("Found XROMM video: " + filename)
                vid_list.append(filename)
                continue
            else:
                continue
    return(vid_list)

def trimVid(vid_list, length, codec='I420'):
    for video in vid_list:
        name = video.split("/")[-1]
        root = video.replace(name,'')
        output_vid = root+"robo_trim_"+name.replace(" ", "")
        cap = cv2.VideoCapture(video)    
        fourcc = cv2.VideoWriter_fourcc(*codec)
        out = cv2.VideoWriter(output_vid,fourcc, 30.0, (1024,1024))
        starttime = dt.now()
        index = 0
        while(cap.isOpened()):
            ret, frame = cap.read()
            if ret==True:
                if index < length:
                    out.write(frame)
                    index += 1
            else:
                break
        endtime = dt.now()
        cap.release()
        out.release()
        print("Trimmed "+output_vid+" to "+str(length)+" frames in {}.".format(endtime-starttime))
        
def compressVid(vid_list, codec='avc1'):
    for video in vid_list:
        name = video.split("/")[-1]
        root = video.replace(name,'')
        output_vid = root+"robo_comp_"+name.replace(" ", "")
        cap = cv2.VideoCapture(video)    
        fourcc = cv2.VideoWriter_fourcc(*codec)
        out = cv2.VideoWriter(output_vid,fourcc, 30.0, (1024,1024))
        starttime = dt.now()
        index = 0
        while(cap.isOpened()):
            ret, frame = cap.read()
            if ret==True:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                clahe = cv2.createCLAHE(clipLimit=3, tileGridSize= (64,64))
                frame = clahe.apply(frame)
                frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)  
                out.write(frame)
            else:
                break
        endtime = dt.now()
        cap.release()
        out.release()
        print("Compressed "+output_vid+" in {}.".format(endtime-starttime))
        
def enhanceVid(vid_list, codec='MJPG'):
    for video in vid_list:
        name = video.split("/")[-1]
        root = video.replace(name,'')
        output_clahe = root+"robo_CLAHE_"+name.replace(" ", "")
        output_eroded = root+"robo_CLAHE_eroded_"+name.replace(" ", "")
        output_dilated = root+"robo_CLAHE_dilated_"+name.replace(" ", "")
        output_opened = root+"robo_CLAHE_opened_"+name.replace(" ", "")
        output_dodge = root+"robo_CLAHE_dodge_"+name.replace(" ", "")
        output_softlight = root+"robo_CLAHE_softlight_"+name.replace(" ", "")
        output_screen = root+"robo_CLAHE_screen_"+name.replace(" ", "")
        output_highlight = root+"robo_CLAHE_highlight_"+name.replace(" ", "")

        cap = cv2.VideoCapture(video)    
        fourcc = cv2.VideoWriter_fourcc(*codec)
        
        out_clahe = cv2.VideoWriter(output_clahe,fourcc, 30.0, (1024,1024))
        out_eroded = cv2.VideoWriter(output_eroded,fourcc, 30.0, (1024,1024))
        out_dilated = cv2.VideoWriter(output_dilated,fourcc, 30.0, (1024,1024))
        out_opened = cv2.VideoWriter(output_opened,fourcc, 30.0, (1024,1024))
        out_dodge = cv2.VideoWriter(output_dodge,fourcc, 30.0, (1024,1024))
        out_softlight = cv2.VideoWriter(output_softlight,fourcc, 30.0, (1024,1024))
        out_screen = cv2.VideoWriter(output_screen,fourcc, 30.0, (1024,1024))       
        out_highlight = cv2.VideoWriter(output_highlight,fourcc, 30.0, (1024,1024))
        
        starttime = dt.now()
        index = 0
        while(cap.isOpened()):
            ret, frame = cap.read()
            if ret==True:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                clahe = cv2.createCLAHE(clipLimit=2, tileGridSize= (64,64))
                frame_clahe = clahe.apply(frame)
            
                kernel3 = np.ones((3,3),np.uint8)
                kernel5 = np.ones((5,5),np.uint8)
                kernel9 = np.ones((9,9),np.uint8)
                kernel17 = np.ones((17,17),np.uint8)
                kernel25 = np.ones((25,25),np.uint8)
                
                frame_dilated3 = cv2.dilate(frame_clahe,kernel3,iterations = 1)
                frame_eroded3 = cv2.erode(frame_clahe,kernel3,iterations = 1)  
                
                frame_eroded5 = cv2.erode(frame_clahe,kernel5,iterations = 1)
                frame_eroded9 = cv2.erode(frame_clahe,kernel9,iterations = 1)
                frame_eroded17 = cv2.erode(frame_clahe,kernel17,iterations = 1)  
                frame_eroded25 = cv2.erode(frame_clahe,kernel25,iterations = 1)
                frame_opened = cv2.dilate(frame_eroded5,kernel3,iterations = 1)
                
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA,4).astype(np.float32)
                frame_clahe = cv2.cvtColor(frame_clahe, cv2.COLOR_BGR2BGRA,4).astype(np.float32)
                frame_eroded5 = cv2.cvtColor(frame_eroded5, cv2.COLOR_BGR2BGRA,4).astype(np.float32)
                frame_eroded9 = cv2.cvtColor(frame_eroded9, cv2.COLOR_BGR2BGRA,4).astype(np.float32)
                frame_eroded17 = cv2.cvtColor(frame_eroded17, cv2.COLOR_BGR2BGRA,4).astype(np.float32)
                frame_eroded25 = cv2.cvtColor(frame_eroded25, cv2.COLOR_BGR2BGRA,4).astype(np.float32)
                
                frame_dodge = blend_modes.dodge(frame_clahe, frame_clahe, 0.5)
                frame_softlight = blend_modes.soft_light(frame,frame_clahe,1)
                frame_screen = blend_modes.screen(frame,frame_clahe,1)
                
                frame_highlight = blend_modes.difference(frame, frame_eroded9, 1)
                frame_highlight = blend_modes.addition(frame_highlight, frame_clahe,1)
                
#                 frame_highlight9 = blend_modes.divide(frame,frame_eroded9,1)
#                 frame_highlight9 = blend_modes.multiply(frame_highlight9,frame_clahe,1)
#                 frame_highlight17 = blend_modes.divide(frame,frame_eroded17,1)
#                 frame_highlight17 = blend_modes.multiply(frame_highlight17,frame_clahe,1)
#                 frame_highlight25 = blend_modes.divide(frame,frame_eroded25,1)
#                 frame_highlight25 = blend_modes.multiply(frame_highlight25,frame_clahe,1)
#                 frame_highlight =  blend_modes.soft_light(frame_highlight9, frame_highlight17, 0.5)
#                 frame_highlight =  blend_modes.soft_light(frame_highlight, frame_highlight25, 0.5)
                
                frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2BGR).astype(np.uint8)
                frame_clahe = cv2.cvtColor(frame_clahe, cv2.COLOR_BGRA2BGR).astype(np.uint8)
                frame_eroded5 = cv2.cvtColor(frame_eroded5, cv2.COLOR_BGRA2BGR).astype(np.uint8)  
                frame_opened = cv2.cvtColor(frame_opened, cv2.COLOR_GRAY2BGR)  
                frame_dodge = cv2.cvtColor(frame_dodge, cv2.COLOR_BGRA2BGR).astype(np.uint8)  
                frame_softlight = cv2.cvtColor(frame_softlight, cv2.COLOR_BGRA2BGR).astype(np.uint8)  
                frame_screen = cv2.cvtColor(frame_screen, cv2.COLOR_BGRA2BGR).astype(np.uint8)  
                frame_highlight = cv2.cvtColor(frame_highlight, cv2.COLOR_BGRA2BGR).astype(np.uint8)  
                
                frame_dilated3 = cv2.cvtColor(frame_dilated3, cv2.COLOR_GRAY2BGR)
                frame_eroded3 = cv2.cvtColor(frame_eroded3, cv2.COLOR_GRAY2BGR)





                out_clahe.write(frame_clahe)
                out_eroded.write(frame_eroded5)
                out_opened.write(frame_opened)
                out_dodge.write(frame_dodge)
                out_softlight.write(frame_softlight)
                out_screen.write(frame_screen)
                out_highlight.write(frame_highlight)
                
                out_dilated.write(frame_dilated3)

                index += 1
#                 print("Working on frame: "+str(index))

            else:
                break
        endtime = dt.now()
        cap.release()
        out_clahe.release()
        out_eroded.release()
        out_opened.release()
        out_dodge.release()
        out_softlight.release()
        out_screen.release()
        out_highlight.release()
        
        out_dilated.release()
        print("Enhanced "+name+" in {}.".format(endtime-starttime))



In [134]:
# list_to_convert = scanDir(input_dir,"robo")
# trimVid(list_to_convert, 100)

list_to_convert = scanDir(input_dir,"robo")
enhanceVid(list_to_convert)


# list_to_compress = scanDir(input_dir)
# compressVid(list_to_compress)





Found XROMM video: /Users/phil/Desktop/Working/Digitizing/23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat/run3circles_90-35-4ms/Camera No.2.avi
Found XROMM video: /Users/phil/Desktop/Working/Digitizing/23Apr18.LaiRegnault.SEP85.LS.biceps_teres_lat/run3circles_90-35-4ms/Camera No.1.avi
Enhanced Camera No.2.avi in 0:09:42.036596.
Enhanced Camera No.1.avi in 0:09:06.844945.


In [45]:
###### IMPROVED WITH TRIM FUNCTION
filename = "robo_trimmed"

def scanDir(directory, filter):
    vid_list=[]
    for root, dirs, files in os.walk(directory):
        for name in files:
            if name.endswith(".avi") and name.find(filter)>=1:
                filename = os.path.join(root, name)
                print("Found XROMM video: " + filename)
                vid_list.append(filename)
                continue
            else:
                continue
    return(vid_list)

def trimVid(vid_list, length, codec='I420'):
    for video in vid_list:
        root = video.replace(video.split("/")[-1],'')
        output_vid = root+"robo_trimmed"+'_'+video[-5:]
        cap = cv2.VideoCapture(video)    
        fourcc = cv2.VideoWriter_fourcc(*codec)
        out = cv2.VideoWriter(output_vid,fourcc, 30.0, (1024,1024))
        starttime = dt.now()
        index = 0
        while(cap.isOpened()):
            ret, frame = cap.read()
            if ret==True:
                if index < length:
                    out.write(frame)
                    index += 1
            else:
                break
        endtime = dt.now()
        cap.release()
        out.release()
        print("Trimmed "+output_vid+" to "+str(length)+" frames in {}.".format(endtime-starttime))
        
def enhanceVid(vid_list, codec='I420'):
    for video in vid_list:
        root = video.replace(video.split("/")[-1],'')
        output_vid = root+"robo_enhanced"+'_'+video[-5:]
        cap = cv2.VideoCapture(video)    
        fourcc = cv2.VideoWriter_fourcc(*codec)
        out = cv2.VideoWriter(output_vid,fourcc, 30.0, (1024,1024))
        starttime = dt.now()
        index = 0
        while(cap.isOpened()):
            ret, frame = cap.read()
            if ret==True:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                frame = cv2.fastNlMeansDenoising(frame,None,3,3,3)
                clahe = cv2.createCLAHE(clipLimit=3, tileGridSize= (64,64))
                frame_clahe = clahe.apply(frame)
                kernel = np.ones((37,37),np.uint8)
                frame_eroded = cv2.erode(frame_clahe,kernel,iterations = 1)
                frame_eroded = cv2.bitwise_not(frame_eroded)

                ###blend-modes requires float32_BGRA data
                frame_clahe_float = cv2.cvtColor(frame_clahe, cv2.COLOR_BGR2BGRA).astype(np.float32)
                frame_eroded_float=cv2.cvtColor(frame_eroded, cv2.COLOR_BGR2BGRA).astype(np.float32)

                blend =  blend_modes.multiply(frame_clahe_float,frame_eroded_float,1)               
                ###cv.write needs uint8_BGR data###
                blend = cv2.cvtColor(blend, cv2.COLOR_BGRA2BGR)
                blend = blend.astype(np.uint8)
                out.write(blend)
                index += 1
#                 print("Working on frame: "+str(index))

            else:
                break
        endtime = dt.now()
        cap.release()
        out.release()
        print("Made "+output_vid+" in {}.".format(endtime-starttime))
        

##31x31kernel invert works well with 1.0 multiply blend and xray COM

In [4]:
##WORKING! CREATES 3 FILES
def scanDir(directory, filter):
    vid_list=[]
    for root, dirs, files in os.walk(directory):
        for name in files:
            if name.endswith(".avi") and name.find(filter) == -1:
                filename = os.path.join(root, name)
                print("Found XROMM video: " + filename)
                vid_list.append(filename)
                continue
            else:
                continue
    return(vid_list)

def makeNewVids(vid_list):
    for video in vid_list:
        root = video.replace(video.split("/")[-1],'')
        v1_filename = root+prepend_v1+'_'+video[-5:]
        v2_filename = root+prepend_v2+'_'+video[-5:]
        v3_filename = root+prepend_v3+'_'+video[-5:]
        enhanceVid(video, v1_filename, 'I420')
        enhanceVid(video, v2_filename, 'I420', erode=1)
        enhanceVid(video, v3_filename, 'jpeg')

def enhanceVid(input_vid, output_vid, codec, erode=0):
    cap = cv2.VideoCapture(input_vid)    
    fourcc = cv2.VideoWriter_fourcc(*codec)
    out = cv2.VideoWriter(output_vid,fourcc, 30.0, (1024,1024))
    starttime = dt.now()
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            if codec=='jpeg':
                out.write(frame)
            else:                
                frame = cv2.fastNlMeansDenoising(frame,None,3,3,3)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                clahe = cv2.createCLAHE(clipLimit=3, tileGridSize= (64,64))
                frame = clahe.apply(frame)
                if erode==1:
                    kernel = np.ones((5,5),np.uint8)
                    frame = cv2.erode(frame,kernel,iterations = 1)
                frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
                out.write(frame)
        else:
            break
    endtime = dt.now()
    cap.release()
    out.release()
    print("Made "+output_vid+" in {}.".format(endtime-starttime))



In [31]:
##TEST: MULTIFRAME DENOISING...try 5,3,2 for params 2 being temporal
## RESULT: NOT MUCH DIFFERENCE, 4X COMPUTING TIME...DONT BOTHER

prepend_v2 = "comp_multiframe_denoise"
# prepend_v2 = "comp_singleframe_denoise"




def scanDir(directory, filter):
    vid_list=[]
    for root, dirs, files in os.walk(directory):
        for name in files:
            if name.endswith(".avi") and name.find(filter) == -1:
                filename = os.path.join(root, name)
                print("Found XROMM video: " + filename)
                vid_list.append(filename)
                continue
            else:
                continue
    return(vid_list)

def makeNewVids(vid_list):
    for video in vid_list:
        root = video.replace(video.split("/")[-1],'')
        v1_filename = root+prepend_v1+'_'+video[-5:]
        v2_filename = root+prepend_v2+'_'+video[-5:]
        enhanceVid(video, v2_filename, raw_codec)
        
def enhanceVid(input_vid, output_vid, codec):
    cap = cv2.VideoCapture(input_vid)    
    fourcc = cv2.VideoWriter_fourcc(*codec)
    out = cv2.VideoWriter(output_vid,fourcc, 30.0, (1024,1024))
    starttime = dt.now()
    index = 0
    buffer = []
    readingframe = []
    while(cap.isOpened()):
        ret,frame = cap.read()
        if ret==True:
            buffer.append(frame)
            if index < 3 or index > vid_frames-2:
                readingframe = [buffer[index],buffer[index],buffer[index]]
            else:
                readingframe = [buffer[index-2],buffer[index-1],buffer[index]]
            frame = cv2.fastNlMeansDenoisingMulti(readingframe, 1, 3, None, 1, 3, 3)

#             frame = cv2.fastNlMeansDenoising(frame,None,3,3,3)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            
            

            clahe = cv2.createCLAHE(clipLimit=3, tileGridSize= (64,64))
            frame = clahe.apply(frame)
#             kernel = np.ones((5,5),np.uint8)
#             frame_eroded = cv2.erode(frame,kernel,iterations = 1)
#             frame_new = cv2.addWeighted(frame,0.6,frame_eroded,0.4,0)

#             frame_new = cv2.cvtColor(frame_new, cv2.COLOR_GRAY2BGR)
            frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
            index += 1
            out.write(frame)
        else:
            break
    endtime = dt.now()
    cap.release()
    out.release()
    print("Made "+output_vid+" in {}.".format(endtime-starttime))



In [97]:
##TEST: COMBINE 2 LAYERS
##RESULT: LOOKS GREAT, DOESN'T TRACK GREAT. VULNERABLE TO OCCLUSION OR NEAR-MISSES

prepend_v2 = "uncomp_erode_stack"


def scanDir(directory, filter):
    vid_list=[]
    for root, dirs, files in os.walk(directory):
        for name in files:
            if name.endswith(".avi") and name.find(filter) == -1:
                filename = os.path.join(root, name)
                print("Found XROMM video: " + filename)
                vid_list.append(filename)
                continue
            else:
                continue
    return(vid_list)

def makeNewVids(vid_list):
    for video in vid_list:
        root = video.replace(video.split("/")[-1],'')
        v1_filename = root+prepend_v1+'_'+video[-5:]
        v2_filename = root+prepend_v2+'_'+video[-5:]
        enhanceVid(video, v2_filename, 'I420')
        
def enhanceVid(input_vid, output_vid, codec):
    cap = cv2.VideoCapture(input_vid)    
    fourcc = cv2.VideoWriter_fourcc(*codec)
    out = cv2.VideoWriter(output_vid,fourcc, 30.0, (1024,1024))
    starttime = dt.now()
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            frame = cv2.fastNlMeansDenoising(frame,None,3,3,3)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            clahe = cv2.createCLAHE(clipLimit=3, tileGridSize= (64,64))
            frame = clahe.apply(frame)
            kernel = np.ones((5,5),np.uint8)
            frame_eroded = cv2.erode(frame,kernel,iterations = 1)
            frame_new = cv2.addWeighted(frame,0.6,frame_eroded,0.4,0)

            frame_new = cv2.cvtColor(frame_new, cv2.COLOR_GRAY2BGR)
            out.write(frame_new)
        else:
            break
    endtime = dt.now()
    cap.release()
    out.release()
    print("Made "+output_vid+" in {}.".format(endtime-starttime))


small blocksize parameter (<15) with adaptive threshold makes faint beads disappear
3-4 are considered extreme values for CLAHE clip limit
tilegridsize in CLAHE must > size of features 
erosion with a 3,3 kernel makes video blocky, helps with sensor noise. don't bother denoising. 2 passes makes big markers, works pretty well with 3-64 clahe. equivalent to 1-pass 5x5
31x31 kernel is about the size of default searchwindow in xmalab
mjpeg is better than h264 for compression quality, similar filesize
denoising: less is better. 2,7,21 is equivalent to 5+,3,3
dilation: doenst work to separate occcluding markers: become too small to track

## denoise tests ##
- 2-3-9 = 3-3-3
- 2-5-13 slightly better
- 21 last param seems bad, blurs details too much 
- 4-9-21 is unusable
- All are better than baseline
- 0-0-0 (Clahe without denoise) is better than baseline, worse than 333?
- 1-3-3 is better than 2-3-3, 3-3-3 better than both
- Test against 2-64x64 CLAHE with 3-3-3:
    - 1-8: worse, similar to baseline
    - 2-16: worse, better than 1-8, strong on dense scapular bead
    - 2.5-24: worse than 2-16
    - 3-16: worse, worse than 2-16
    - 4-16: on par with 2-16
- Test against 2-64x64 CLAHE with 3-3-3:
    - 1-128: much worse
    - 2-128: also worse
    - 2-96: bad: moving tiles
Adaptive threshold test:
- Gaussian > mean


## speed tests
### cervical bead:
- baseline: 8.33fps
- compressed: 5.95fps
- CLAHEerode: 8.33fps
- CLAHE: 8.08fps
### distal radius (to f300):
- CLAHEerode: 3.33fps
- CLAHE: 0.44fps
- baseline: 2.5fps



In [102]:
###### TEST: COMBINE 2 LAYERS (INVERT)
## RESULT: LOOKS COOL BUT DOESN"T REALLY HELP

prepend_v2 = "uncomp_erode_stack_invert_add"


def scanDir(directory, filter):
    vid_list=[]
    for root, dirs, files in os.walk(directory):
        for name in files:
            if name.endswith(".avi") and name.find(filter) == -1:
                filename = os.path.join(root, name)
                print("Found XROMM video: " + filename)
                vid_list.append(filename)
                continue
            else:
                continue
    return(vid_list)

def makeNewVids(vid_list):
    for video in vid_list:
        root = video.replace(video.split("/")[-1],'')
        v1_filename = root+prepend_v1+'_'+video[-5:]
        v2_filename = root+prepend_v2+'_'+video[-5:]
        enhanceVid(video, v2_filename, 'I420')
        
def enhanceVid(input_vid, output_vid, codec):
    cap = cv2.VideoCapture(input_vid)    
    fourcc = cv2.VideoWriter_fourcc(*codec)
    out = cv2.VideoWriter(output_vid,fourcc, 30.0, (1024,1024))
    starttime = dt.now()
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame = cv2.fastNlMeansDenoising(frame,None,3,3,3)
            clahe = cv2.createCLAHE(clipLimit=3, tileGridSize= (64,64))
            frame = clahe.apply(frame)
            kernel = np.ones((9,9),np.uint8)
            frame_eroded = cv2.erode(frame,kernel,iterations = 1)
            frame_eroded = cv2.bitwise_not(frame_eroded)
            frame_eroded = cv2.equalizeHist(frame_eroded).astype("float32")
#             frame_new = cv2.addWeighted(frame,0.75,frame_eroded,0.25,0)
            
            frame_new = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGBA, 4)
            frame_eroded = cv2.cvtColor(frame_eroded, cv2.COLOR_GRAY2RGBA,4)
            frame_new = blend_modes.multiply(frame_new, frame_eroded, 1)




            
            out.write(frame_new)
        else:
            break
    endtime = dt.now()
    cap.release()
    out.release()
    print("Made "+output_vid+" in {}.".format(endtime-starttime))



In [ ]:
%%time
cap = cv2.VideoCapture(input_path)

# Define the codec and create VideoWriter object
## working fourcc codecs per https://gist.github.com/takuma7/44f9ecb028ff00e2132e:
## 'avc1' = h.264; 'jpeg' = jpeg; 'mp4v' = mpeg4; 'raw' = uncompressed; 'tiff' = tiff, semi-working
fourcc = cv2.VideoWriter_fourcc(*'I420')
out = cv2.VideoWriter(output_path,fourcc, 30.0, (1024,1024))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        frame = cv2.fastNlMeansDenoising(frame,None,2,3,3)
        kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
        frame = cv2.filter2D(frame, -1, kernel)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        clahe = cv2.createCLAHE(clipLimit=2, tileGridSize= (64,64))
        frame = clahe.apply(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
        # write the flipped frame
        out.write(frame)

#         cv2.imshow('frame',frame)
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
    else:
        break

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

In [4]:
##LUCAS-KANADE TRACKER
cap_lk = cv2.VideoCapture(output_path)
# Create old frame
_, frame = cap_lk.read()
old_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
# Lucas kanade params
lk_params = dict(winSize = (8, 8),
                 maxLevel = 4,
                 criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.03))
# Mouse function
def select_point(event, x, y, flags, params):
    global point, point_selected, old_points
    if event == cv2.EVENT_LBUTTONDOWN:
        point = (x, y)
        point_selected = True
        old_points = np.array([[x, y]], dtype=np.float32)
cv2.namedWindow("Frame")
cv2.setMouseCallback("Frame", select_point)
point_selected = False
point = ()
old_points = np.array([[]])
while True:
    _, frame = cap_lk.read()
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if point_selected is True:
        cv2.circle(frame, point, 5, (0, 0, 255), 2)
        new_points, status, error = cv2.calcOpticalFlowPyrLK(old_gray, gray_frame, old_points, None, **lk_params)
        old_gray = gray_frame.copy()
        old_points = new_points
        x, y = new_points.ravel()
        cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1)
    if key == 27:
        break
cap_lk.release()
cv2.destroyAllWindows()

In [ ]:
#OTHER TRACKERS
# import the necessary packages
from imutils.video import VideoStream
import argparse
import imutils
import time
import cv2


# initialize a dictionary that maps strings to their corresponding
# OpenCV object tracker implementations
OPENCV_OBJECT_TRACKERS = {
    "csrt": cv2.TrackerCSRT_create,
    "kcf": cv2.TrackerKCF_create,
    "boosting": cv2.TrackerBoosting_create,
    "mil": cv2.TrackerMIL_create,
    "tld": cv2.TrackerTLD_create,
    "medianflow": cv2.TrackerMedianFlow_create,
    "mosse": cv2.TrackerMOSSE_create
}
 
# initialize OpenCV's special multi-object tracker
trackers = cv2.MultiTracker_create()

vs = cv2.VideoCapture(output_path)
tracker_choice = "csrt"
    
# loop over frames from the video stream
while True:
    # grab the current frame, then handle if we are using a
    # VideoStream or VideoCapture object
    frame = vs.read()
    _, frame = vs.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
 
    # check to see if we have reached the end of the stream
    if frame is None:
        break
 
    # resize the frame (so we can process it faster)
#     frame = imutils.resize(frame, width=600)
    
    
    # grab the updated bounding box coordinates (if any) for each
    # object that is being tracked
    (success, boxes) = trackers.update(frame)

    # loop over the bounding boxes and draw then on the frame
    for box in boxes:
        (x, y, w, h) = [int(v) for v in box]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the 's' key is selected, we are going to "select" a bounding
    # box to track
    if key == ord("s"):
        # select the bounding box of the object we want to track (make
        # sure you press ENTER or SPACE after selecting the ROI)
        box = cv2.selectROI("Frame", frame, fromCenter=False,
            showCrosshair=True)

        # create a new object tracker for the bounding box and add it
        # to our multi-object tracker
        tracker = OPENCV_OBJECT_TRACKERS[tracker_choice]()
        trackers.add(tracker, frame, box)
        
    # if the `q` key was pressed, break from the loop
    elif key == ord("q"):
        break


# otherwise, release the file pointer

vs.release()

# close all windows
cv2.destroyAllWindows()

In [2]:
#OTHER TRACKERS
import cv2
import sys
  
if __name__ == '__main__' :
 
    # Set up tracker.
    # Instead of MIL, you can also use
 
    tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
    tracker_type = tracker_types[]
 
    if tracker_type == 'BOOSTING':
        tracker = cv2.TrackerBoosting_create()
    if tracker_type == 'MIL':
        tracker = cv2.TrackerMIL_create()
    if tracker_type == 'KCF':
        tracker = cv2.TrackerKCF_create()
    if tracker_type == 'TLD':
        tracker = cv2.TrackerTLD_create()
    if tracker_type == 'MEDIANFLOW':
        tracker = cv2.TrackerMedianFlow_create()
    if tracker_type == 'GOTURN':
        tracker = cv2.TrackerGOTURN_create()
    if tracker_type == 'MOSSE':
        tracker = cv2.TrackerMOSSE_create()
    if tracker_type == "CSRT":
        tracker = cv2.TrackerCSRT_create()
 
    # Read video
    video = cv2.VideoCapture(input_path)
 
    # Exit if video not opened.
    if not video.isOpened():
        print("Could not open video")
        sys.exit()
 
    # Read first frame.
    ok, frame = video.read()
    if not ok:
        print('Cannot read video file')
        sys.exit()
     
    # Define an initial bounding box
#     bbox = (287, 23, 86, 320)
 
    # Uncomment the line below to select a different bounding box
    bbox = cv2.selectROI(frame, False)
 
    # Initialize tracker with first frame and bounding box
    ok = tracker.init(frame, bbox)
 
    while True:
        # Read a new frame
        ok, frame = video.read()
        if not ok:
            break
         
        # Start timer
        timer = cv2.getTickCount()
 
        # Update tracker
        ok, bbox = tracker.update(frame)
 
        # Calculate Frames per second (FPS)
        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);
 
        # Draw bounding box
        if ok:
            # Tracking success
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
        else :
            # Tracking failure
            cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
 
        # Display tracker type on frame
        cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2);
     
        # Display FPS on frame
        cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);
 
        # Display result
        cv2.imshow("Tracking", frame)
 
        # Exit if ESC pressed
        k = cv2.waitKey(1) & 0xff
        if k == 27 : break

In [78]:
#TESTING CLAHE PARAMETERS
img = cv2.imread('/Users/phil/Desktop/clahetest/cam2occluding/clahe_0.png',0)
base_path = '/Users/phil/Desktop/clahetest/cam2occluding/clahe_0.png'

thresholds = [0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0]
tile_sizes = [2,4,8,16,24,32,40,48,56,64,72,80]
for threshold in thresholds:
    for tile in tile_sizes:
        clahe =  cv2.createCLAHE(clipLimit=threshold, tileGridSize=(tile,tile))
        cl = clahe.apply(img)
        cv2.imwrite(base_path+'clahe'+str(threshold)+'_'+str(tile)+'.jpg',cl)

- original is 100f in 11s
- avc1 is slooow: 100f in 1:21
- jpeg is faster than avc1: 100f in 23s, 5s encode
- tiff: 12s encode, width/height mismatch
- raw is 5s encode, width/height mismatch
- I420 tracks as fast as raw, 5s encode, 50% bigger file
- 64tile_single 1,3,3: 26. no sharper than 2,3,3. more false positives
- 64tile_single 2,3,3: 26s. looks a lot like 3,3,3, slightly sharper markers. good compromise
- 64tile_single 3,3,3: 26s. Good denoise on meat, no effect on bg
- 64tile_single 3,5,3: 26s. No change compared to 3,3,3
- 64tile_single 3,3,5: 54s. slightly better than 3,5,3
- 64tile_single 3,3,11: 3mins exactly. slightly better than 3,3,5, but hard to say exactly
- 64tile_single 4,3,3: 26. similar to 3,3,3. too blurry to track well
- 64tile_single 5,3,3: 32s. points blurrier than 4,3,3. bad
- denoised+clahe tracks better than raw
- combo to beat is CLAHE 2,64 | denoise 2,3,3
- unsharp mask makes things worse

- good xmalab settings with clahe denoise: threshold 8, xray center of mass
- pre or post denoise makes no difference?